The following are the states of the Kalman filter:

\begin{equation}
x = 
\begin{bmatrix} 
p \\
q \\
r \\
a_x \\
a_y \\
a_z \\
\phi \\
\theta \\
\psi \\
TAS \\
m_y \\
m_z 
\end{bmatrix}
\end{equation}



| State Var | Description            | Units   |
|:-----------|:------------------------|:---------:|
| $p$      | Body axis roll rate    | rad/sec |
| $q$      | Body axis pitch rate   | rad/sec |
| $r$      | Body axis yaw rate     | rad/sec |
| $a_x$     | Body axis acceleration | m/s$^2$ |
| $a_y$     | Body axis acceleration | m/s$^2$ |
| $a_z$     | Body axis acceleration | m/s$^2$ |
| $\phi$    | Euler roll angle       | rad |
| $\theta$  | Euler bank angle       | rad |
| $\psi$    | Euler yaw angle        | rad |
| $TAS$     | True airspeed          | m/s |
| $m_x$     | Earth mag vector       | $\mu$T |
| $m_z$     | Earth mag vector       | $\mu$T |

Note that by definition the y component of earth's  field ($m_y$) is zero. In other words, the definition used here is that the earth coordinate system x points north.

The number of states is $n_{states}$ and in this case is equal to 12.

There are a number of other parameters used throughout this analysis and they are described here:

| Variable | Description | Units |
| :-- | :-- | :--: |
| g | Gravitational acceleration | m/s |

The Euler angle rotation angles above ($\phi$, $\theta$, and $\psi$) are used to create a rotation matrix to convert from earth coordinates to body frame coordinates.  TODO: The analysis should extend to use a quaternion to avoid gimbal lock problems.

$$
H_{EB} = 
\begin{bmatrix}
\cos\theta cos\psi & \cos\theta sin\psi & -sin\theta \\
\sin\phi \sin\theta - cos\phi sin\psi & sin\phi sin\theta sin\psi + \cos\phi \cos\psi & sin\phi cos\theta \\
\cos\phi  \sin\theta  \cos\psi + \sin\phi  \sin\psi & \cos\phi \sin\theta \sin\psi - \sin\phi \cos\psi & \cos\phi \cos\theta \\
\end{bmatrix}
$$


Kalman Filter parameters:
    
| Parameter | Description |    
| :--- | :--- |
| $x$ | State vector ($n_{states} \times 1$ ) |
| $P$ | Covariance of states ($n_{states} \times n_{states}$) |
| $Q$ | Covariance of the prediction step. Can be thought of as how incorrect is the model used in the prediction step. ($n_{states} \times n_{states}$) |
| $F$ | For a standard Kalman filter, this is the matrix that when multiplied by $x$ generates the model prediction. ($n_{states} \times n_{states}$) |
| $R$ | Covariance of sensors ($n_{sensors} \times n_{sensors}$) |
| $H$ | Maps $x$ to sensor space. ($n_{sensors} \times n_{states}$) |

Prediction:
$$
x_{k | k-1} = F x_{k-1 | k-1} 
$$
$$ P = F P F^T + Q $$

Update:
$$ 
\begin{align}
y_k &= z_k - H x_{k | k-1} \\
S &= H P H^T + R \\
K &= P H^T S^{-1} \\
x_{k|k} &= x_{k | k-1} + K y \\
P &= (I - K H) P \\ 
\end{align}
$$ 


In [6]:
from sympy import *
import numpy as np
from DCM_sym import HEB
init_printing()

In [7]:
# Create all state variables
droll, dpitch, dyaw = symbols(['p', 'q', 'r'])
phi, theta, psi = symbols(['phi', 'theta', 'psi'])
ax, ay, az = symbols(['ax', 'ay', 'az'])
magx, magy, magz = symbols(['magx', 'magy', 'magz'])
magxe, magze = symbols(['magxe', 'magze'])
TAS = symbols('TAS')
g = symbols('g')
dt = symbols('dt')
nstates = 12
statevars = np.array([droll, dpitch, dyaw, ax, ay, az, phi, theta, psi, TAS, magxe, magze])

The model used for this filter is an assumption of coordinated flight.  Using this assumption, we can calculate the [rate of rotation](https://en.wikipedia.org/wiki/Standard_rate_turn) and [centripital acceleration](https://en.wikipedia.org/wiki/Centripetal_force) due to this rotation.  

The rate of rotation simplifies to:
$$ \dot{\phi} = \tan\left( \frac{\phi g}{TAS} \right)  $$

## Compass caibration

### Hard Iron

This effect can be thought of as the offset of the magnetic vector tracing a perfect circle.  It is corrected by simplying subtracting a bias from the raw sensor values.

### Soft Iron

Soft iron is the distortion of that perfect circle.

A [generic model](https://www.vectornav.com/support/library/magnetometer) for compensation is

$$ 
M = 
\begin{bmatrix}
C_1 & C_2 & C_3 \\
C_4 & C_5 & C_6 \\
C_7 & C_8 & C_9 \\
\end{bmatrix} 
\begin{bmatrix}
H_x - C_{10} \\
H_y - C_{11} \\
H_z - C_{12} \\
\end{bmatrix}
$$